In [67]:
# Import necessary modules
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import Adam

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import tslearn
from tslearn.datasets import UCR_UEA_datasets
from tslearn.preprocessing import TimeSeriesScalerMeanVariance, TimeSeriesResampler, TimeSeriesScalerMinMax

In [68]:
dataset_name = 'ElectricDevices'

In [69]:
# Load and preprocess the time series data
X_train, y_train, X_test, y_test = UCR_UEA_datasets().load_dataset(dataset_name)

In [70]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((8926, 96, 1), (8926,), (7711, 96, 1), (7711,))

In [71]:
#normalize the data
X_train = TimeSeriesScalerMinMax().fit_transform(X_train)
X_test = TimeSeriesScalerMinMax().fit_transform(X_test)

In [72]:
# Resample the data to 128 time steps
X_train = TimeSeriesResampler(sz=128).fit_transform(X_train)
X_test = TimeSeriesResampler(sz=128).fit_transform(X_test)

In [73]:
# Convert the data to torch tensors
X_train = torch.from_numpy(X_train).float()
X_test = torch.from_numpy(X_test).float()
y_train = torch.from_numpy(y_train).long()
y_test = torch.from_numpy(y_test).long()

In [74]:
#start class from 0
y_train = y_train - 1
y_test = y_test - 1

In [75]:
#Datasets
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
#Dataloaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [76]:
# Define the CNN model
class CNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(CNN, self).__init__()
        self.sequential = nn.Sequential(
            nn.Conv1d(in_channels=input_size, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.AvgPool1d(kernel_size=2, stride=2),
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            #flatten
            nn.Flatten(),
            nn.LazyLinear(out_features=128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(in_features=128, out_features=num_classes)
        )
    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.sequential(x)
        return x

In [80]:
# Define the LSTM model
class LSTM(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=64, num_layers=2, batch_first=True)
        self.fc = nn.Linear(in_features=64, out_features=num_classes)
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.fc(x[:, -1, :])
        return x

In [83]:
# Define the combined CNN-LSTM model
class CNN_LSTM(nn.Module):
    def __init__(self, input_size, num_classes):
        super(CNN_LSTM, self).__init__()
        self.cnn = CNN(input_size, num_classes)
        self.lstm = LSTM(input_size, num_classes)
        self.fc = nn.Linear(in_features=2*num_classes, out_features=num_classes)
    def forward(self, x):
        x1 = self.cnn(x)
        x2 = self.lstm(x)
        x = torch.cat((x1, x2), dim=1)
        x = self.fc(x)
        return x

In [90]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [92]:
#cnn
input_size = X_train.shape[-1]
num_classes = len(np.unique(y_train))
model_cnn = CNN(input_size, num_classes).to(device)
#lstm
model_lstm = LSTM(input_size, num_classes).to(device)
#cnn_lstm
model_cnn_lstm = CNN_LSTM(input_size, num_classes).to(device)

In [94]:
from typing import List

In [110]:
def train(models:List, train_loader:DataLoader, epochs:int):
    criterion = nn.CrossEntropyLoss()
    for model in models:
        print("Training model: ", model.__class__.__name__)
        model.train()
        optimizer = Adam(model.parameters(), lr=0.001)
        for epoch in range(epochs):
            for i, (x, y) in enumerate(train_loader):
                x = x.to(device)
                y = y.to(device)
                optimizer.zero_grad()
                y_pred = model(x)
                loss = criterion(y_pred, y)
                loss.backward()
                optimizer.step()
                if (i+1) % 10 == 0:
                    print(f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')
        print("Training completed for model: ", model.__class__.__name__)
    #save models
    torch.save(model_cnn.state_dict(), 'model_cnn.pth')
    torch.save(model_lstm.state_dict(), 'model_lstm.pth')
    torch.save(model_cnn_lstm.state_dict(), 'model_cnn_lstm.pth')
    print('Models saved!')

In [111]:
#train
models = [model_cnn, model_lstm, model_cnn_lstm]
train(models, train_loader, epochs=1)

Training model:  CNN
Epoch [1/1], Step [10/140], Loss: 0.7387
Epoch [1/1], Step [20/140], Loss: 0.8917
Epoch [1/1], Step [30/140], Loss: 0.7619
Epoch [1/1], Step [40/140], Loss: 0.6439
Epoch [1/1], Step [50/140], Loss: 0.6534
Epoch [1/1], Step [60/140], Loss: 0.8272
Epoch [1/1], Step [70/140], Loss: 0.8498
Epoch [1/1], Step [80/140], Loss: 1.1424
Epoch [1/1], Step [90/140], Loss: 0.8256
Epoch [1/1], Step [100/140], Loss: 0.6804
Epoch [1/1], Step [110/140], Loss: 0.7685
Epoch [1/1], Step [120/140], Loss: 0.9463
Epoch [1/1], Step [130/140], Loss: 0.9444
Epoch [1/1], Step [140/140], Loss: 0.7558
Training completed for model:  CNN
Training model:  LSTM
Epoch [1/1], Step [10/140], Loss: 1.4430
Epoch [1/1], Step [20/140], Loss: 1.4108
Epoch [1/1], Step [30/140], Loss: 1.4844
Epoch [1/1], Step [40/140], Loss: 1.8868
Epoch [1/1], Step [50/140], Loss: 1.6929
Epoch [1/1], Step [60/140], Loss: 1.4708
Epoch [1/1], Step [70/140], Loss: 1.5746
Epoch [1/1], Step [80/140], Loss: 1.2849
Epoch [1/1], St

In [121]:
#test
def test(models, test_loader):
    with torch.no_grad():
        correct = 0
        total = 0
        for model in models:
            model.eval()
            for x, y in test_loader:
                x = x.to(device)
                y = y.to(device)
                y_pred = model(x)
                _, predicted = torch.max(y_pred.data, 1)
                total += y.size(0)
                correct += (predicted == y).sum().item()
            print(f'Accuracy of the {model.__class__.__name__} model on the test set: {100 * correct / total:.2f} %')

In [122]:
test(models, test_loader)

Accuracy of the CNN model on the test set: 63.27 %
Accuracy of the LSTM model on the test set: 46.34 %
Accuracy of the CNN_LSTM model on the test set: 52.86 %
